# What are Subroutines?

- Let's consider a sample program:-

<img src="images/sample-prog-1.png">

<img src="images/sample-prog-2.png">

- This program reads and adds two digits between 0 and 4. It validates each input byte and writes an error message if a byte is invalid. It works as follows:-
<br><br>
    - The GETN1 routine reads and validates the first input digit. It loops until a valid digit is obtained. SAVEN1 is a part of GETN1.
<br><br>
    - The GETN2 routine reads and validates the second input digit. It also loops until a valid digit is obtained.
<br><br>
    - The ADDIT routine adds the two digits and writes the answer. Control is returned to GETN1 to start the next problem.
<br><br>
    - The NEWLIN routine starts a new line on the terminal.
<br><br>
    - The CHECK routine validates the byte in A. If the input byte is between '0' and '4', the X register is set to 0. If not, it is set to \\$FF. NOGOD is part of the check routine.
<br><br>
    - The ERROR routine puts out a message if the input doesn't validate. ERLOOP is part of the routine.
<br><br>
    - The INPUT routine reads one byte from the keyboard and places it in the accumulator. TESTIN is part of this routine.
<br><br>
    - The OUTPUT routine prints one byte from the accumulator to the terminal.
<br><br>
- Most assembly language programs contain three major parts:-
<br><br>
    - The main line is the code that's logically between the start (the first executed instruction) and the stop (however that happens). If the main line contains more than one routine (as indicated by labelled statements), the routine receives control by fall-through or branches.
<br><br>
    - The subroutines are sections of code that are not in the main line. They recieve control only by JSR commands. They are positioned and coded so that they never receive control by fall-through, jumps, or branches.
<br><br>
    - The data area definitions reserve memory bytes to hold data. They are positioned so that they will never receive control.
<br><br>
- In the above program lines 6-33 comprise of main line, 34-72 contain subroutines, and 73-75 comprise of data area definitions.
<br><br>
- A subroutine is thus a routine that receives control only by the JSR instruction. It mostly performs one function and releases control by a return which returns control to the instruction following JSR.
<br><br>
- Using JSR to transfer control to subroutine is called calling the subroutine. In other assembly languages this command to transfer control is CALL. This works in two steps:-
<br><br>
    - The address in the PC register is pushed back into the stack. At the time it is pushed it is pointing at the third byte of the JSR instruction.
<br><br>
    - The address in the JSR instruction operand is loaded into the PC. Thus, that address becomes the next instruction.
<br><br>
- Return works as follows:-
<br><br>
    - The top of the stack is pulled into the PC. This should be the address that was pushed by the JSR instruction.

# Coding a subroutine

- Three characteristics of a good subroutine are:-
<br><br>
    - Complete and accurate.
<br><br>
    - No side effects.
<br><br>
    - At least one return.

# Preserving Original Values

- A subroutine avoids side effects by returning the registers and the stack in exactly the same condition as it receives them. It may use those but at the end it should restore them to their original values.
<br><br>
- The exception is any area that is expected to be affected by a subroutine (e.g. INPUT affects accumulator and status byte register). OUTPUT subroutine doesn't affect the accumulator.

## Questions

1) Code a routine that has a subroutine that affects A and X registers. Both of them affect status register. The subroutine should return in the original condition:-

```assembly
PHP
PHA
TXA
PHA
...    ; CODE FOR SUBROUTINE
PLA
TAX
PLA
PLP
```

2) Write instructions to save and restore the status, X, and Y registers. Use data locations for storage wherever possible:-

```assembly
PHP
STX SAVEX
STY SAVEY
....    ; CODE FOR SUBROUTINE
LDX SAVEX
LDY SAVEY
PLP
```

- Here the order of restoration of X and Y doesn't matter but status register should be restored last, or it may be changed by the other operations.

## Note

- It is very important to pull things which were pushed from the stack as the return address is expected at top of stack to return control. If it is unbalanced the remainder of the program will not work.

# Returning from a Subroutine

- A subroutine returns control when it reaches a return instruction. The operand code is RTS (ReTurn from Subroutine). There is no operand and no flags are affected.

## Questions

1) Code instructions to return the value of X from the stack, then return control to the calling routine:-

```assembly
PLA
TAX
RTS
```

2) Code a subroutine called ECHO that reads a byte into the accumulator, echoes it, and stores it in memory at INCHAR. Leave all the registers intact when you return control to the calling routine:-

```assembly
ECHO    EQU  *
        PHP
        PHA
        JSR  INPUT
        JSR  OUTPUT
        STA  INCHAR
        PLA
        PLP
        RTS
```

3) Code a subroutine that reads a byte from the terminal. If the input byte is less than \\$20, return control. If the byte is \\$20 or more, move it to the X register and then return control. Note that the contents of the X register are intended to be changed by this subroutine and should not be preserved:-

```assembly
LETWTY    EQU  *
          PHP
          PHA
          JSR  INPUT
          JSR  OUTPUT
          CLC
          SBC  #$20
          BPL  MOVEX
          PLA
          PLP
          RTS
MOVEX     EQU  *
          TXA
          RTS
```

4) Code a subroutine called STALL that writes out the message: PLEASE WAIT -- I'M THINING. (Use two single quotes to store one single quote):-

```assembly
STALL    EQU  *
         PHP
         PHA
         STX  SAVEX
         LDX  #0
OUTLOP   EQU  *
         LDA  WAITMS,X
         JSR  OUTPUT
         INX
         CPX  #27
         BNE  OUTLOP
         LDX  SAVEX
         PLA
         PLP
         RTS
WAITMS   ASC  'PLEASE WAIT -- I''M THINKING'
SAVEX    DS   1
```


5) Code a routine that adds 5 to the contents of register A. If a carry results, reset the register to zero. Otherwise, just return control. It is not necessary to preserve the contents of any registers in the subroutine:-

```assembly
INC5    EQU  *
        CLC
        ADC  #5
        BCC  ENDING
        LDA  #0
ENDING  RTS
```

# Passing Data

- Many subroutines require data to be passed to them. These values are called passed data.

## Questions

1) Suppose a subroutine is called that expects to print out one of several possible messages. All the possible messages are stored one after the other in a data area called 'TEXT'. The subroutine expects to find the length of the current message in the Y register. It expects to use X register to find the first byte of the actual message. Code a set of instructions that will call this subroutine (named MESOUT) for a 15-character message starting at TEXT+9:-

```assembly
LDY #15
LDX #9
JSR MESOUT
```

# I/O Subroutines

- One major problem with I/O is that the microprocessor is way faster than most I/O devices.
<br><br>
- When writing an input subroutine we have to wait till the input device has a byte available.
<br><br>
- When writing an output subroutine we have to wait for the output device to get ready for one byte.
<br><br>
- So, most of the time an I/O subroutine spends time stalling, pausing, waiting, spinning its wheels, slowing down the computer.
<br><br>
- There are connections between the I/O device and the microprocessor at two memory addresses. One location contains I/O data. The other one is for status information. The status information tells whether the device has an input byte to be read or is ready to receive an output byte. 
<br><br>
- \\$C000 is the memory location for data from the keyboard. The keyboard sends standard seven-bit ASCII characters to this location. When a character arrives, a signal turns on the high-order bit. This means that \\$C000 has a value of 128 or more whenever a byte of input data has arrived there.  So in an input subroutine the high-order bits have to be checked and data has to be moved from input byte to accumulator.
<br><br>
- Any reference to location \\$C010 in the system will turn off the high-order bit in \\$C000. It does not matter what command we use here, in the above sample, we have used STA.
<br><br>
- One problem in this method is that the input characters don't match the standard ASCII characters.

## Questions

1) A basic subroutine has an output location (e.g. \\$5000) and an output status byte (e.g. \\$5050), which is set to 128 when the output device is ready, 0 otherwise. Code an appropriate output subroutine:-

```assembly
OUTPUT    EQU  *
          STX  SAVEX
          TAX
          LDA  #%10000000
TEST      EQU  *
          BIT  $5050
          BEQ  TEST
          TXA              ; RESTORE 'A' REGISTER
          STX  $5000
          LDX  #$0
          STX  $5050
          LDX  SAVEX
          RTS
```

- Most systems come with pre-loaded I/O subroutines. In case of a monitor the location where byte is to be printed is also to be taken care of in the output subroutine.
<br><br>
- One caution while using system subroutines is that we have to be careful of the memory location they use and not use them with program's instruction and data area.

# Program Design

- One way is to write all the main logic in various subroutines and use the main line only to call these subroutines, but the overhead is tremendous as a call and return can take four times as long to execute as a jump instruction, also the pushes and pulls to stack add to the total time and also use up memory space.
<br><br>
- Most application programs should thus use many subroutines to make their code readable, while system programs which demand faster execution times and lesser memory should contain lesser subroutines.
<br><br>
- A situation where subroutines should be called in system programs (and application programs too) is when the code is repeating.
<br><br>
- Another reason for using subroutines is when the same subroutine appears in many programs.

# Exercise Programs

## Question 1

- This subroutine writes out a message from the data area TEXT to an output device. The displacement for the first byte of the message is passed in the X register. The length of the message is passed in the Y register. Call the OUTPUT routine to actually write each byte:-

```assembly
WRITE    EQU  *
         PHP
         PHA
OUTLOP   EQU  *
         LDA  TEXT,X
         JSR  OUTPUT
         INX
         DEY
         BNE  OUTLOP
         PLA
         PLP
         RTS
```

## Question 2

- This subroutine reads, echos, and validates an incoming byte. A byte is valid if it is an upper case letter. If invalid, write the message WRONG -- TRY AGAIN. Continue reading until a valid byte is obtained. Leave the byte in the accumulator and return control. Use INPUT and OUTPUT subroutines to read and write single bytes. Call the subroutine you wrote for question 1 to write the message:-

```assembly
UPPERA    EQU  $41
ZPLUS1    EQU  $5B
GETLET    EQU  *
          STY  SAVEX
TRYONE    EQU  *
          JSR  INPUT
          JSR  OUTPUT
          CMP  #UPPERA
          BMI  ERROR
          CMP  #ZPLUS1
          BMI  OK
ERROR     EQU  *
          LDY  MESLNG
          LDX  #0
          JSR  WRITE
          JMP  GETLET
OK        EQU  *
          LDY  SAVEY
          RTS
SAVEY     DS   1
MESLNG    DFB  18
TEXT      ASC  'WRONG -- TRY AGAIN'
```

## Question 3

- This routine - not a subroutine - reads and stores an incoming message in memory. It should loop until the letter Z is read. That terminates the message (Store the Z). Each character of the incoming message should be read, echoed, and validated using the subroutine you wrote for question 2:-

```assembly
          LDX  #0
READIN    EQU  *
          STX  SAVEX
          JSR  GETLET
          LDX  SAVEX
          STA  INMES,X
          INX
          CMP  #'Z'
          BMI  READIN
DONE      JMP  DONE
INMES     DS   80
```

## Question 4

- This subroutine writes one character from the accumulator on a printer. Use these features:-
<br><br>
    - Status byte at address \\$C50F.
<br><br>
    - MSB indicates output status, on for ready.
<br><br>
    - Data byte at address \\$C50A.
   
```assembly
PRINT    EQU  *
         STX  SAVEX
         PHA
         LDA  #%10000000
TEST     EQU  *
         BIT  $C50F
         BEQ  TEST
         PLA
         STA  $C50A
         LDX  #0
         STX  $C50F
         LDX  SAVEX
         RTS
```

## Question 5

- This subroutine reads one character from a keyboard. The terminal status byte is at \\$DD1C. The first and second bits (LSB and LSB + 1) are on when a byte is ready to be read. The data byte is at address \\$DD1D. Put the character in the accumulator:-

```assembly
INPUT    EQU  *
         STX  SAVEX
         LDA  #%00000011
TEST     EQU  *
         BIT  $DD1C
         BEQ  TEST
         LDA  $DD1D
         PHP
         LDX  #0
         STX  $DD1C
         LDX  SAVEX
         PLP
         RTS
```

## Question 6

- This subroutine prints text on the printer by calling the print subroutine you wrote for question 4. After each character is printed, check for an input byte from the terminal described in question 5 above. If any byte is input, discontinue printing. The following data is passed to a subroutine: (a) the beginning displacement of the text is in register X; (b) the length of the text is in register Y:-

```assembly
PTEX    EQU  *
        LDA  TEXT,X
        JSR  PRINT
        LDA  #%00000011
        BIT  $DD1C
        BNE  ENDSUB
        INX
        DEY
        BNE  PTEX
ENDSUB  EQU  *
        RTS
```

## Question 7

- This routine causes the following message to be printed, preceded by form feed and carriage return:-
<br><br>
    NOW IS THE TIME FOR ALL GOOD PEOPLE TO COME TO THE AID OF THEIR PARTY.
<br><br>
- Use the subroutine you wrote for question 6 to print the text:-

```assembly
NOWPRN    EQU  *
          LDY  #72
          LDX  #0
          JSR  PTEX
DONE      JMP  DONE
TEXT      DBF  $0D, $0A
          ASC  'NOW IS THE TIME FOR ALL '
          ASC  'GOOD PEOPLE TO COME TO THE '
          ASC  'AID FOR THEIR PARTY.'
```